### Testing the transcribing of the audio file using Whisper models

In [1]:
import whisper

In [2]:
# Load the model
model_base = whisper.load_model("base")  # options: tiny, base, small, medium, large
model_medium = whisper.load_model("medium")  # options: tiny, base, small, medium, large
model_turbo = whisper.load_model("turbo")  # options: tiny, base, small, medium, large


KeyboardInterrupt: 

In [ ]:
import pickle
result_base= model_base.transcribe("/Users/joaomata/Desktop/LBMP/E-DAIC/600_P/600_AUDIO.wav")
with open("result_base_600.pkl", "wb") as f:
    pickle.dump(result_base, f)

/Users/joaomata/Desktop/LBMP/.venv/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
result_medium = model_large.transcribe("/Users/joaomata/Desktop/LBMP/E-DAIC/600_P/600_AUDIO.wav")
with open("result_medium.pkl", "wb") as f:
    pickle.dump(result_medium, f)

100%|█████████████████████████████████████| 1.42G/1.42G [01:03<00:00, 24.0MiB/s]
/Users/joaomata/Desktop/LBMP/.venv/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Python(97011) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [ ]:
result_turbo = model_turbo.transcribe("/Users/joaomata/Desktop/LBMP/E-DAIC/600_P/600_AUDIO.wav")
with open("result_turbo.pkl", "wb") as f:
    pickle.dump(result_turbo, f)

/Users/joaomata/Desktop/LBMP/.venv/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Python(97610) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [ ]:
# Print the transcription
print("Result (Base)")  
print(result_base["text"])
print("Result (Large)")
print(result_medium["text"])
print("Result (turbo)")
print(result_turbo["text"])

### Using DepRoBERTa to classify transcripts

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("rafalposwiata/deproberta-large-depression")
model = AutoModelForSequenceClassification.from_pretrained("rafalposwiata/deproberta-large-depression")

/Users/joaomata/Desktop/LBMP/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import pickle

# Load texts from pickle files
with open("result_base_600.pkl", "rb") as f:
    result_base = pickle.load(f)
with open("result_medium.pkl", "rb") as f:
    result_medium = pickle.load(f)
with open("result_turbo.pkl", "rb") as f:
    result_turbo = pickle.load(f)

texts = {
    "Base": result_base["text"],
    "Medium": result_medium["text"],
    "Turbo": result_turbo["text"]
}

labels = ["No Depression", "Moderate Depression", "Depression"]

for model_name, text in texts.items():
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=1)
        pred_idx = torch.argmax(probs).item()
    print(f"\nModel: {model_name}")
    print(f"Text: {text}") 
    print(f"Logits: {logits}")
    print(f"Probabilities: {probs}")
    print(f"Predicted class: {pred_idx} ({labels[pred_idx]})")

 Okay. So you're in the bridge, you're in the hoppy, up here for just a second, and I'll go ahead and shrink her back down. Okay. So there she is. So I'm going to go ahead and shrink her back down while I continue setting some things up. Okay. Come on. Okay. Okay. Are you okay with this? Yes. Okay. So, are you going to do it today? I'm fine. Let's go. Where are you from originally? Let's go. Really? Yes. Where are you from originally? Where are you from originally? The weather. Where are you from originally? The crime. I see what you mean. Do you consider yourself a man? I don't know, Demise. Sometimes when I'm feeling tense, I turn on the fist and scream saber. Hey, I know it's not polite, but it's the best I've got. What do you do to relax? Rhyp poetry. What are some things that make you really not? What happens to your mind? Nothing. What do you do when you're in a lot of it? Listen to music. How are you going to control your tongue? Very good. When was the last time you heard this?